[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/romiaprilian7406/sp500-ebitda-margin/blob/main/notebooks/sp500_ebitda_dataset_clean.ipynb)

# Import Libraries

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import time
import warnings
import os

warnings.filterwarnings('ignore')

# Ticker list Function

In [2]:
# Mengambil daftar ticker S&P 500 terbaru dari dataset publik
def get_sp500_tickers():
    try:
        url = 'https://raw.githubusercontent.com/datasets/s-and-p-500-companies/refs/heads/main/data/constituents.csv'
        df = pd.read_csv(url)
        # Mengubah format ticker (misal: BRK.B -> BRK-B) agar sesuai dengan yfinance
        tickers = [t.replace('.', '-') for t in df['Symbol'].tolist()]
        print(f"Berhasil mendapatkan {len(tickers)} ticker S&P 500")
        return tickers
    except Exception as e:
        print(f"Gagal mengambil ticker: {e}")
        return []

# Data Fetching Function

In [3]:
def fetch_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        # Skip jika data kosong atau tidak punya Revenue
        if not info or 'totalRevenue' not in info or info['totalRevenue'] is None:
            return None

        # 1. Data Balance Sheet (Prioritas: Laporan Audit Terakhir)
        bs_data = {}
        try:
            bs = stock.balance_sheet
            if not bs.empty:
                latest_bs = bs.iloc[:, 0] # Tahun terbaru

                bs_data['total_debt'] = (
                    latest_bs.get('Total Debt') or
                    latest_bs.get('Total Liabilities Net Minority Interest')
                )
                bs_data['total_cash'] = (
                    latest_bs.get('Cash And Cash Equivalents') or
                    latest_bs.get('Cash') or
                    latest_bs.get('Cash Cash Equivalents And Short Term Investments')
                )
        except:
            pass

        # 2. Penyusunan Dictionary
        data = {
            'ticker': ticker,
            'sector': info.get('sector', 'Unknown'),

            # Komponen Skala (Scale)
            'revenue': info.get('totalRevenue'),

            # Komponen Pertumbuhan (Growth)
            # Diambil dari info karena butuh data TTM vs Prior Year
            'revenue_growth': info.get('revenueGrowth'),

            # Komponen Struktural (Untuk Rasio Aman)
            # Prioritas Balance Sheet, Fallback ke Info
            'total_debt': bs_data.get('total_debt') or info.get('totalDebt'),
            'total_cash': bs_data.get('total_cash') or info.get('totalCash'),

            'gross_profit': info.get('grossProfits') or info.get('grossProfit'),
            # Atau ambil langsung margin jika ada:
            'gross_margin_reported': info.get('grossMargins'),

            # TARGET VARIABLE (Raw)
            # Margin dihitung di tahap cleaning
            'ebitda': info.get('ebitda'),
            'ebitda_margin_reported': info.get('ebitdaMargins') # Sebagai pembanding/backup
        }

        return data

    except Exception as e:
        return None

# Simple Data Cleaning & Preprocessing

In [4]:
def clean_data(df):
    df = df.copy()

    # 1. Drop Duplicates (Entity Integrity)
    if 'ticker' in df.columns:
        df = df.drop_duplicates(subset=['ticker'], keep='first')

    # 2. Type Casting (Memastikan angka adalah Float)
    num_cols = ['revenue', 'revenue_growth', 'total_debt', 'total_cash', 'gross_profit', 'gross_margin_reported', 'ebitda', 'ebitda_margin_reported']

    for col in num_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 3. Handle System Artifacts
    df = df.replace([np.inf, -np.inf], np.nan)

    # 4. Standardize Sector
    if 'sector' in df.columns:
        df['sector'] = df['sector'].fillna('Unknown').astype(str).str.strip().str.title()
        df = df[df['sector'] != 'Unknown']

    # 5. SANITY CHECK WAJIB (Data Validity)
    # Revenue wajib ada dan positif
    df = df.dropna(subset=['revenue'])
    df = df[df['revenue'] > 0]

    # 6. MEMBUAT TARGET VARIABLE (Wajib dilakukan di awal untuk Labeling)
    # Hitung EBITDA Margin di sini agar dataset yang disimpan sudah punya LABEL (y)

    # Hitung manual: EBITDA / Revenue
    if 'ebitda' in df.columns:
        df['ebitda_margin'] = df['ebitda'] / df['revenue']

    # Fallback ke reported margin jika hitungan manual NaN
    if 'ebitda_margin_reported' in df.columns:
        df['ebitda_margin'] = df['ebitda_margin'].fillna(df['ebitda_margin_reported'])

    # Drop baris tanpa Target
    df = df.dropna(subset=['ebitda_margin'])

    # FEATURE ENGINEERING (Di tahap Cleaning minimal)
    # Gross Margin sebagai fitur input
    if 'gross_profit' in df.columns and 'revenue' in df.columns:
        df['gross_margin'] = df['gross_profit'] / df['revenue']

    # Fallback ke reported
    if 'gross_margin_reported' in df.columns:
        df['gross_margin'] = df['gross_margin'].fillna(df['gross_margin_reported'])

    # 7. Final Column Selection (Simpan RAW data + Target)
    final_cols = [
        'ticker', 'sector',
        'revenue', 'revenue_growth',
        'total_debt', 'total_cash',
        'gross_margin', 'ebitda_margin',

    ]

    cols_to_keep = [c for c in final_cols if c in df.columns]
    return df[cols_to_keep]

# Main Execution

In [5]:
tickers = get_sp500_tickers()
data_list = []

for ticker in tqdm(tickers):
    stock_data = fetch_data(ticker)
    if stock_data:
        data_list.append(stock_data)
    time.sleep(0.5)

Berhasil mendapatkan 503 ticker S&P 500


  0%|          | 0/503 [00:00<?, ?it/s]

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: WBA"}}}


In [6]:
# Buat DataFrame
df_raw = pd.DataFrame(data_list)

if not df_raw.empty:
    # 1. Bersihkan Dataset (Fungsi ini butuh kolom 'ticker' lowercase)
    df = clean_data(df_raw)

    print(f"Total Raw Data  : {len(df_raw)}")
    print(f"Total Clean Data: {len(df)}")

    # Preview
    print("\nContoh Data:")
    display(df.head())
else:
    print("Gagal mengumpulkan data atau data kosong")
    df_final = pd.DataFrame()

Total Raw Data  : 502
Total Clean Data: 502

Contoh Data:


,ticker,sector,revenue,revenue_growth,total_debt,total_cash,gross_margin,ebitda_margin
0,MMM,Industrials,24824999936,0.035,1.365900e+10,5.600000e+09,0.409789,0.248177
1,AOS,Industrials,3830099968,0.044,2.167000e+08,2.396000e+08,0.385134,0.204773
2,ABT,Healthcare,43842998272,0.069,1.502100e+10,7.616000e+09,0.562598,0.267933
3,ABBV,Healthcare,59643998208,0.091,6.714400e+10,5.524000e+09,0.714607,0.494920
4,ACN,Technology,69672976384,0.073,8.182866e+09,1.147873e+10,0.319140,0.175427


# Simple EDA

In [7]:
# Shape
rows, cols = df.shape
print(f"Dimensi Dataset")
print(f"Total Baris    : {rows}")
print(f"Total Kolom    : {cols}\n")

# Info
print(df.info(),"\n")

# Definisi Fitur & Target
target_col = 'ebitda_margin'

# Fitur adalah semua kolom selain Target dan Metadata
features_list = [col for col in df.columns if col not in [target_col]]

print(f"Jumlah Fitur : {len(features_list)}")
print(f"Daftar Fitur : {features_list}\n")

# Statistik Deskriptif Fitur Numerik
print(f"\nStatistik Deskriptif")
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

try:
    display(df[numeric_cols].describe().T.round(2))
except:
    print(df[numeric_cols].describe().T.round(2))

# Missing Values
print(f"\nLaporan Missing Values:")
missing_count = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Jumlah Hilang': missing_count,
    'Persentase (%)': missing_pct
})

# Filter hanya yang ada missing value & Sort dari yang terbanyak
missing_only = missing_df[missing_df['Jumlah Hilang'] > 0].sort_values(by='Jumlah Hilang', ascending=False)

# Format persentase setelah sorting
missing_only['Persentase (%)'] = missing_only['Persentase (%)'].map('{:.2f}%'.format)

if not missing_only.empty:
    print(missing_only)
else:
    print("Tidak ada missing values")

Dimensi Dataset
Total Baris    : 502
Total Kolom    : 8

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ticker          502 non-null    object 
 1   sector          502 non-null    object 
 2   revenue         502 non-null    int64  
 3   revenue_growth  500 non-null    float64
 4   total_debt      498 non-null    float64
 5   total_cash      502 non-null    float64
 6   gross_margin    502 non-null    float64
 7   ebitda_margin   502 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 31.5+ KB
None 

Jumlah Fitur : 7
Daftar Fitur : ['ticker', 'sector', 'revenue', 'revenue_growth', 'total_debt', 'total_cash', 'gross_margin']


Statistik Deskriptif


,count,mean,std,min,25%,50%,75%,max
revenue,502.0,3.607808e+10,7.414990e+10,7.723950e+08,6.809425e+09,1.411000e+10,2.904350e+10,7.030610e+11
revenue_growth,500.0,1.000000e-01,2.100000e-01,-4.500000e-01,3.000000e-02,7.000000e-02,1.200000e-01,3.190000e+00
total_debt,498.0,2.005513e+10,4.157584e+10,4.530000e+05,3.474648e+09,8.469321e+09,1.913300e+10,4.543110e+11
total_cash,502.0,7.456578e+09,3.163555e+10,1.326000e+06,5.418695e+08,1.528644e+09,3.966500e+09,4.693170e+11
gross_margin,502.0,4.900000e-01,2.400000e-01,-1.080000e+00,3.200000e-01,4.700000e-01,6.400000e-01,1.000000e+00
ebitda_margin,502.0,2.600000e-01,1.900000e-01,-1.460000e+00,1.400000e-01,2.400000e-01,3.600000e-01,9.200000e-01



Laporan Missing Values:
                Jumlah Hilang Persentase (%)
total_debt                  4          0.80%
revenue_growth              2          0.40%


# Export Dataset

In [9]:
file_name = 'sp500_ebitda_dataset.csv'
df.to_csv(file_name, index=False)
print(f"File tersimpan: {file_name}")

File tersimpan: sp500_ebitda_dataset.csv
